# Module demonstration

In [1]:
!pip install git+https://github.com/xrustle/multi_label_classifier -q

In [2]:
import os
import pandas as pd
from sklearn.metrics import jaccard_score
from sklearn.model_selection import train_test_split

from multi_label_classifier import MultiLabelClassifier

#### Загрузка данных

In [3]:
EXCEL_FILE = 'ceo_train_v2.01.xlsx'
if not os.path.isfile(EXCEL_FILE):
    !wget https://github.com/xrustle/multi_label_classifier/raw/master/tests/test_data/ceo_train_v2.01.xlsx

In [4]:
df = pd.read_excel('ceo_train_v2.01.xlsx')
df = df[['Комментарий', 'target']]

X_train, X_test, y_train, y_test = train_test_split(
    df['Комментарий'],
    df['target'],
    test_size=0.2,
    random_state=42
)

#### Обучение и предсказание

Лучше иметь предустановленный XGBoost для GPU для значительного ускорения обучения

In [5]:
%%time
model = MultiLabelClassifier(
    n_estimators=3,  # Число ChainClassifier в ансамбле
    var_threshold=5e-5
)
model.fit(X_train, y_train, silent=False)

y_pred = model.predict(X_test)  # Предсказания в виде one-hot векторов
y_pred_labels = model.predict_labels(X_test)  # Текстовые предсказания с классами через запятую
y_pred[0]

Data preprocessing...
Training...



CPU times: user 4min 2s, sys: 18 s, total: 4min 20s
Wall time: 2min 14s


array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False])

In [6]:
model.mlb.transform([['empty']])[0]==1

array([False, False, False, False,  True, False, False, False, False,
       False, False, False, False])

In [7]:
y_true = model.mlb.transform(y_test.apply(lambda x: x.split(',')))

#### Строим предсказание в виде векторов для оценки

Для оценки используем Jaccard score. Она показывает насколько сильно совпадают реальные наборы классов с предсказанными.<br> Чем ближе к 1, тем лучше.<br> https://en.wikipedia.org/wiki/Jaccard_index

In [8]:
jaccard_score(y_true, y_pred, average='samples').round(decimals=3)

0.967

#### Пример предсказания

In [9]:
df = pd.DataFrame({
    'X_test': X_test, 
    'y_test': y_test, 
    'y_pred_labels': y_pred_labels,
    'y_true': y_true.tolist(),
    'y_pred': y_pred.tolist()
})
df[['X_test', 'y_test', 'y_pred_labels']]

,X_test,y_test,y_pred_labels
30490,за сегодняшний день зафиксировано 2 случая уде...,empty,empty
17284,прошу перезагрузить банкомат. отсутствуют опер...,"remote_reboot,no_oper","no_oper,remote_reboot"
1109,просьба перезагрузить,remote_reboot,remote_reboot
5536,просьба опросить банкомат. нет операций,no_oper,no_oper
31356,"загружается по внутренним, банкомат завис. на ...",empty,empty
...,...,...,...
21913,просьба удаленно перезагрузить и вывести кассе...,"remote_reboot,cassette","cassette,remote_reboot"
10406,прошу проверить ус нет операций.,no_oper,no_oper
18134,добрый день!просьба удаленно перезагрузить ус....,"remote_reboot,cassette","cassette,remote_reboot"
31838,поступила жалоба о плохой работе карт-ридера -...,empty,empty


#### Пример ошибок

In [10]:
df[df.y_true != df.y_pred][['X_test', 'y_test', 'y_pred_labels']]

,X_test,y_test,y_pred_labels
10430,нет операций. что с ус?,no_oper,"coordination_eta,no_oper"
33913,прошу указать сроки востановительных работ,coordination_eta,empty
9273,"жалоба клиента: ""атм зависает, при попытке сня...",cards,empty
30516,проблема с картридером. у инкассаторов из 3 ба...,empty,cards
29940,05.12.2019 01:50 карта 4966 ус изъял карту пос...,empty,cards
...,...,...,...
14764,"20.07.2019 18:40 атм изъял банковскую карту, х...",cards,empty
7525,"банкомат зажевывает все карты - деньги отдает,...",cards,empty
30739,повторно.поступают жалобы на медленную обработ...,empty,cards
8135,атм изъяла карту и не выдал,cards,empty


Классы cards и empty сильно коллерируют между собой. Но я их и как человек не всегда правильно угадываю)